In [1]:
from sshtunnel import SSHTunnelForwarder
import psycopg2 as psy
import pandas as pd
from IPython.display import FileLink
import geopandas as gpd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import paramiko
from io import StringIO
from shapely.geometry import MultiPoint, MultiPolygon
from sklearn import preprocessing, cluster
import scipy
import scipy.cluster
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from shapely.ops import unary_union
import calendar
from datetime import datetime
import base64

/home/rajat/anaconda3/envs/gisenv/lib/python3.11/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
def get_conn(SSH_required,key_path):   #for getting a connection as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = []
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        print('Tunnel Started')
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn

In [3]:
def get_df_from_sql(SSH_required, query,key_path):   #for getting a datafarame as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = None
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        # ssh_tunnel._server_list[0].block_on_close = False
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        ssh_tunnel.stop()
        return df_results
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        return df_results

In [4]:
# Usage with the actual path to the private key
SSH_required = 'Yes'
key_path = '/home/rajat/Downloads/tunnel-ssh .cer'
query = """select
    l.awb
,   l.attribution
,   l.lost_date
,   l.shipment_value
,   o.*
,   n.node_name
,   sort_codes
from public.lost_attribution as l
left join public.ops_main as o on l.awb = o.awb
left join application_db.node as n on trim(split_part(n.sort_codes,'/',2)) = trim(o.last_mile_hub)
where shipping_partner = 'Hyperlocal' and created_date >= '2024-01-01'"""


# Establish a connection
conn = get_conn(SSH_required, key_path)

# Retrieve data into a DataFrame
df = get_df_from_sql(SSH_required, query, key_path)

# Now you can perform further operations with the DataFrame 'df'
print(df)



Tunnel Started
Connection Made
                awb attribution   lost_date  shipment_value            awb  \
0      GS1843640610       Rider  2024-02-01           698.0   GS1843640610   
1      GS1752437944          FM  2024-02-01           549.0   GS1752437944   
2      GS2140144762       Rider  2024-02-01           183.0   GS2140144762   
3      GS1832490284       Rider  2024-02-01          1449.0   GS1832490284   
4      GS1641639613         Hub  2024-02-01           463.0   GS1641639613   
...             ...         ...         ...             ...            ...   
1872  BZPPL00342143         Hub  2024-05-21          1851.0  BZPPL00342143   
1873  BZPPL00339109         Hub  2024-05-21          1847.0  BZPPL00339109   
1874  BZPPL00341501         Hub  2024-05-21          3417.0  BZPPL00341501   
1875  BZPPL00339698         Hub  2024-05-21           638.0  BZPPL00339698   
1876  BZPPL00338307         Hub  2024-05-21          1326.0  BZPPL00338307   

      shipment_id          shop_

In [5]:
# Reading GeoJSON
gdf_geojson = gpd.read_file("/home/rajat/GIS Blitznow/India_Pincodes/india_pincodes.shp")

# If the current CRS is geographic, re-project to UTM (EPSG:32644)
if gdf_geojson.crs.is_geographic:
    gdf_geojson = gdf_geojson.to_crs('EPSG:32644')

In [6]:
# Calculating centroids of pincodes
gdf_geojson = gdf_geojson.to_crs(epsg=4326)

gdf_geojson['latitude'] = gdf_geojson['geometry'].centroid.y
gdf_geojson['longitude'] = gdf_geojson['geometry'].centroid.x

# Ensure the pincode column datatype is consistent
df['shipping_pincode'] = df['shipping_pincode'].astype(str)
gdf_geojson['pincode'] = gdf_geojson['pincode'].astype(str)

# Merge GeoDataFrame and DataFrame
merged_gdf = gdf_geojson.merge(df, left_on='pincode', right_on='shipping_pincode')
merged_gdf['last_scan_time'] = pd.to_datetime(merged_gdf['last_scan_time'])
merged_gdf['last_scan_week'] = merged_gdf['last_scan_time'].dt.to_period('W').dt.week

# Generate a list of unique week start dates
week_start_dates = merged_gdf['last_scan_time'].dt.to_period('W').dt.start_time.unique()

# Sort the week start dates in ascending order
week_start_dates = sorted(week_start_dates)

# Generate marks with week numbers as keys and corresponding start dates as labels
marks = {str(week): {'label': str(date.date())} for week, date in enumerate(week_start_dates, start=1)}

min_week = min(int(week) for week in marks.keys())
max_week = max(int(week) for week in marks.keys())

In [7]:
# Initialize Dash app
app = Dash(__name__)

# Define app layout
app.layout = html.Div([
    html.H1("Orders lost on Shipping Pincode", style={'text-align': 'center'}),
    dcc.Input(
        id='search_shippingcity',
        type='text',
        placeholder='Search Shipping City...',
        value='Bangalore',
        style={'width': "40%"}
    ),
    html.Div(style={'height': '20px'}),
    dcc.RangeSlider(
        id='week-slider',
        min=min_week,
        max=max_week,
        marks=marks,
        value=[min_week + 1, min_week+2] 
    ),
    html.Div(style={'height': '20px'}),
    dcc.Dropdown(
        id='attribution-dropdown',
        options=[
            {'label': 'Rider', 'value': 'Rider'},
            {'label': 'Hub', 'value': 'Hub'}
        ],
        value=['Hub', 'Rider'],
        multi=True  # Allow multiple selection
    ),
    html.Div(style={'height': '20px'}),
    html.Div(id='output_container_city', children=[]),
    html.Div(id='output_container_month', children=[]),
    html.Br(),
    html.Div([
        dcc.Graph(id='map', figure={}),
    ])
])

# Callback to update the graph
@app.callback(
    [Output(component_id='output_container_city', component_property='children'),
     Output(component_id='map', component_property='figure')],
    [Input(component_id='search_shippingcity', component_property='value'),
     Input(component_id='week-slider', component_property='value'),
     Input(component_id='attribution-dropdown', component_property='value')]
)
def update_graph(city_slctd, selected_weeks, selected_attributions, min_week=min_week, max_week=max_week, marks=marks):
    if not city_slctd or city_slctd not in merged_gdf['shipping_city'].unique():
        return "", {}
    output_container_city = "Results are shown for the Shipping City: {}".format(city_slctd)
    
    # Convert the selected weeks integers to strings
    selected_min_date = str(selected_weeks[0])
    selected_max_date = str(selected_weeks[1])
    
    # Filter the data for the selected city, selected weeks, and selected attributions
    dff = merged_gdf[(merged_gdf["shipping_city"] == city_slctd) & 
                     (merged_gdf["last_scan_week"].astype(str) >= selected_min_date) &
                     (merged_gdf["last_scan_week"].astype(str) < selected_max_date) &
                     (merged_gdf['attribution'].isin(selected_attributions))]
    
    grouped_data = dff.groupby(['shipping_pincode', 'officename', 'geometry']).size().reset_index(name='lost_orders')
    grouped_data = gpd.GeoDataFrame(grouped_data, geometry='geometry')
    
    # Generate centroid
    centroid = grouped_data.geometry.centroid
    centroid_wgs84 = centroid.to_crs(epsg=4326)
    grouped_data['geometry'] = grouped_data['geometry'].simplify(tolerance=0.001)
    
    fig = px.choropleth_mapbox(
        grouped_data,  # Use filtered data from merged_gdf
        geojson=grouped_data.geometry,  # Pass geometry from filtered data
        locations=grouped_data.index,  # Use index from filtered data
        color='lost_orders',
        hover_data=['shipping_pincode', 'officename', 'lost_orders'],
        mapbox_style="carto-positron",
        center={"lat": centroid_wgs84.y.mean(), "lon": centroid_wgs84.x.mean()},
        zoom=8.5,
        opacity=0.9,
        template='plotly_dark',
        color_continuous_scale="RdYlGn_r"
    )
    return output_container_city, fig


if __name__ == '__main__':
    port = 8055
    app.run(debug=True, port=port, mode='external')
    dash_app_link = f"http://localhost:{port}/"
    print("Link to access Dash app externally:")
    print(dash_app_link)


Link to access Dash app externally:
http://localhost:8055/
